# SIMP Topology Optimization using Gridap in Julia


In the following code: 

$p$ = densities 

$pf$ = filtered densities

$pth$ = thresholded densities 

### Load required packages 

In [ ]:
t1 = time()

In [ ]:
# FEA 
using  Gridap
using  Gridap.Geometry
using  Gridap.Fields
using  Gridap.TensorValues 
using  Gridap.CellData 

# Meshing 
using  Gmsh 
using  GridapGmsh

using  LinearAlgebra 

# Gardient calculation 
using  ChainRulesCore, Zygote
import ChainRulesCore: rrule 

# For MMA 
using  NLopt 

# For plotting 
# using CairoMakie, GridapMakie 

### Constant parameters 
Material properties, such as modulus of elasticity and Poisson's ratio. 
The SIMP penalty is generally assumed as 3~5. 

In [ ]:
const  E_mat = 1.0
const  ν_mat = 0.3 
const  penal = 3; 

### Create the mesh or load already created mesh file using gmsh 
A GUI is also available to use gmsh 

In [ ]:
# Create the mesh
h = 0.009
Lₚ = 0.233
Hₚ = 0.05

gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)

p1 = gmsh.model.geo.addPoint(-Lₚ, -Hₚ, 0.0, h)
p2 = gmsh.model.geo.addPoint(Lₚ, -Hₚ, 0.0, h)
p3 = gmsh.model.geo.addPoint(Lₚ, Hₚ, 0.0, h)
p4 = gmsh.model.geo.addPoint(-Lₚ, Hₚ, 0.0, h)

l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p1)

c1 = gmsh.model.geo.addCurveLoop([l1,l2,l3,l4])
ps1 = gmsh.model.geo.addPlaneSurface([c1])

g4 = gmsh.model.addPhysicalGroup(1, [l4])
g9 = gmsh.model.addPhysicalGroup(1, [l1])
g10 = gmsh.model.addPhysicalGroup(1, [l3])
g8 = gmsh.model.addPhysicalGroup(1, [l2])
pg1 = gmsh.model.addPhysicalGroup(2, [ps1])

gmsh.model.setPhysicalName(2, pg1, "Domain")
gmsh.model.setPhysicalName(1, g4, "DirichletLeft")
gmsh.model.setPhysicalName(1, g9, "ElectricPotentialLeft")
gmsh.model.setPhysicalName(1, g10, "ElectricPotentialRight")
gmsh.model.setPhysicalName(1, g8, "LoadLine")

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
gmsh.write("PiezoBeam.msh")
gmsh.finalize()

model = GmshDiscreteModel("PiezoBeam.msh")

### Constitutive matrix (1 for plane stress, 2 for plane strain) 
Elasticity tensor, constitutive relation, and SIMP penalty relation are formulated here. 

In [ ]:
function  ElasFourthOrderConstTensor(E,ν,PlanarState)
    # 1 for  Plane  Stress  and 2 Plane  Strain  Condition
    if  PlanarState  == 1
        C1111 =E/(1-ν*ν)
        C1122 = (ν*E)/(1-ν*ν)
        C1112 = 0.0
        C2222 =E/(1-ν*ν)
        C2212 = 0.0
        C1212 =E/(2*(1+ν))
    elseif  PlanarState  == 2
        C1111 = (E*(1-ν*ν))/((1+ν)*(1-ν-2*ν*ν))
        C1122 = (ν*E)/(1-ν-2*ν*ν)
        C1112 = 0.0
        C2222 = (E*(1-ν))/(1-ν-2*ν*ν)
        C2212 = 0.0
        C1212 =E/(2*(1+ν))
    end
    C_ten = SymFourthOrderTensorValue(C1111 ,C1112 ,C1122 ,C1112 ,C1212 ,C2212 ,C1122 ,C2212 ,C2222)
    return   C_ten
end 

# Piezoelectric constants
const d₁₁t = -6.98e-3
const d₂₁t = 13.84e-3
const d₂₂t = 13.44e-3
const d₂₃t = 13.04e-3
const a₁₁t = 6e-9
const a₂₂t = 5.47e-9

function PiezoThirdOrderConstTensor(d₁₁,d₂₁,d₂₂,d₂₃)
    e111 = d₁₁
    e112 = d₁₁
    e121 = d₂₁
    e122 = d₂₂
    e211 = d₁₁
    e212 = 0.0
    e221 = d₂₂
    e222 = d₂₃   
    vals = zeros(2,2,2)
    vals[1,:,:] .= [e111 e112; e121 e122]
    vals[2,:,:] .= [e211 e212; e221 e222]
    e_ten = ThirdOrderTensorValue(vals...)
    return e_ten
end

# The σfun function calculates the stress σ given a strain ε using the elasticity tensor C_mat
# function σfun(ε)
#     σ = C_mat⊙ε
#     return  σ
# end 

# Constitutive relations
σ_elas(ε) = C_mat ⊙ ε
σ_piezo(∇ϕ) = D_mat ⋅ ∇ϕ
D_piezo(∇ϕ) = A_mat ⋅ ∇ϕ
D_elas(ε) = D_mat ⋅² ε

#Em(p) effective modulus
function Em(p)
    Em = p ^ penal
    return Em
end 

const  C_mat = ElasFourthOrderConstTensor(E_mat ,ν_mat ,1);
const D_mat = PiezoThirdOrderConstTensor(d₁₁t, d₂₁t, d₂₂t, d₂₃t)
const A_mat = TensorValue(a₁₁t, 0.0, 0.0, a₂₂t)

### Finite Element Analysis 

In [ ]:
# order = 1
# reffe_Disp = ReferenceFE(lagrangian ,VectorValue{2,Float64},order)
# V0_Disp = TestFESpace(model,reffe_Disp;conformity =:H1,
#     dirichlet_tags = ["LeftSupport"],
#     dirichlet_masks =[(true,true)])
# uh = zero(V0_Disp)
# U0_Disp = V0_Disp 

# degree = 2*order
# Ω= Triangulation(model)
# dΩ= Measure(Ω,degree) 

# FE spaces
order = 1
reffe_Disp = ReferenceFE(lagrangian, VectorValue{2,Float64}, order)
V0_Disp = TestFESpace(model, reffe_Disp;
    conformity=:H1,
    dirichlet_tags=["DirichletLeft"],
    dirichlet_masks=[(true,true)])

uApp1(x) = VectorValue(0.0, 0.0)
uApp2(x) = VectorValue(20000, 0.0)
U_Disp = TrialFESpace(V0_Disp,[uApp1,uApp2])

reffe_ElecPot = ReferenceFE(lagrangian, Float64, order)
V0_ElecPot = TestFESpace(model, reffe_ElecPot;
    conformity=:H1,
    dirichlet_tags=["ElectricPotentialLeft"],
    dirichlet_masks=[true])

ElecF = 0.0
phiApp = (ElecF/10)*(1)*1e3
U_ElecPot = TrialFESpace(V0_ElecPot,[phiApp])

V0 = MultiFieldFESpace([V0_Disp, V0_ElecPot])
U = MultiFieldFESpace([U_Disp, U_ElecPot])

# Measures
degree = 2*order
Ω = Triangulation(model)
dΩ = Measure(Ω, degree)

# Defines the elasticity tensor C_mat for plane stress.
# Sets up the finite element spaces (V0_Disp, U0_Disp) for displacement, and the measure dΩ for integration over the domain.


labels = get_face_labeling(model)
LoadTagId = get_tag_from_name(labels ,"LoadLine")
Γ_Load = BoundaryTriangulation(model ;tags = LoadTagId)
dΓ_Load = Measure(Γ_Load ,degree)
n_Γ_Load = get_normal_vector(Γ_Load) 

# Retrieves boundary labels and sets up the load boundary condition and normal vector for the load boundary

p_reffe = ReferenceFE(lagrangian, Float64, 0)
Q = TestFESpace(Ω, p_reffe, vector_type = Vector{Float64})
P = Q
np = num_free_dofs(P) 

pf_reffe = ReferenceFE(lagrangian, Float64, 1)
Qf = TestFESpace(Ω, pf_reffe, vector_type = Vector{Float64})
Pf = Qf 

# Sets up the test function spaces (Q, P, Qf, Pf) for the optimization problem.

fem_params = (;V0_Disp, U_Disp, V0_ElecPot, U_ElecPot, V0, U, Q, P, Qf, Pf, np, Ω, dΩ, dΓ_Load)

# MatrixA is δR/δu
A_Disp(u,v,pth) =  ((p->Em(p))∘pth) * (ε(v) ⊙ (σ_elas∘(ε(u))))
function MatrixA(pth; fem_params)
    A_mat = assemble_matrix(fem_params.U0_Disp, fem_params.V0_Disp) do u, v
        ∫(A_Disp(u,v,pth))fem_params.dΩ
    end
    return lu(A_mat)
end

f = VectorValue(0,-1.0) # Load vector 

function  stepDisp(fem_params,pth)
    A_Disp(u,v,pth) = ((p->Em(p))∘pth) * ε(v) ⊙ (σ_elas∘(ε(u)))
    a_Disp((u, ϕ), (v, ψ)) = ∫((A_Disp(u,v,pth)) + ((∇(v) ⊙ (σ_piezo ∘ (∇(ϕ))))) - (∇(ψ) ⋅ (D_piezo ∘ (∇(ϕ)))) + (∇(ψ) ⋅ (D_elas∘(ε(u)))) + (ρ * coeff_a_u * (v ⋅ u)))fem_params.dΩ 
    b_Disp((v, ψ)) = ∫(v ⋅ f)fem_params.dΓ_Load
    op_Disp = AffineFEOperator(a_Disp ,b_Disp ,fem_params.U0_Disp ,fem_params.V0_Disp)
    res = Gridap.solve(op_Disp)
    uh_out, ϕh = res
    return  get_free_dof_values(uh_out)
end 

# A_Disp function defines the bilinear form for the δR/δu.
# MatrixA function assembles and factorizes the δR/δu matrix.
# stepDisp function solves for the displacement field under the given load.

# MatrixOf is K
function MatrixOf(fem_params)

    return assemble_matrix(fem_params.U0_Disp, fem_params.V0_Disp) do u, v
             0.5*∫((∇(u))' ⊙ (C_mat ⊙ ∇(v)))fem_params.dΩ
    end
end
@show size(MatrixOf(fem_params)); 


# MatrixOf is K function assembles and factorizes the global stiffness matrix.

### Filtering and thresholding 
Helmholtz PDE-based filter is used here. 

(see: Lazarov, Boyan Stefanov, and Ole Sigmund. "Filters in topology optimization based on Helmholtz‐type differential equations." International Journal for Numerical Methods in Engineering 86.6 (2011): 765-781.) 

A smooth approximation of the Heaviside step function is used as the thresholding function. $\beta$ determines the strength of the approximation, and $\eta$ determines the location. Note that $\eta=0.5$ is standard in TO. 

In [ ]:
elemsize = 0.3
r = (3*elemsize)/(2*sqrt(3)) #0.025           # Filter radius
β = 4                       # β∈[1,∞], threshold sharpness
η = 0.5                     # η∈[0,1], threshold center

a_f(r, u, v) = r^2 * (∇(v) ⋅ ∇(u))

function Filter(p0; r, fem_params)
    ph = FEFunction(fem_params.P, p0)
    op = AffineFEOperator(fem_params.Pf, fem_params.Qf) do u, v
        ∫(a_f(r, u, v))fem_params.dΩ + ∫(v * u)fem_params.dΩ, ∫(v * ph)fem_params.dΩ
      end
    pfh = solve(op)
    return get_free_dof_values(pfh)
end

# Filter will give \del

function Threshold(pfh; β, η)
    return  ((tanh(β * η) + tanh(β * (pfh - η))) / (tanh(β * η) + tanh(β * (1.0 - η)))) 

end 

NO_FIELDS = ZeroTangent() 

Dptdpf(pf, β, η) = β * (1.0 - tanh(β * (pf - η))^2) / (tanh(β * η) + tanh(β * (1.0 - η))) # Gradient of thresholding function 
DEdpf(pf, β, η)= penal * ((Threshold(pf; β, η)) ^ (penal-1)) * Dptdpf(pf, β, η) # Gradient of density^penal
DAdpf(u, v, pfh; β, η) = ((p->DEdpf(p, β, η)) ∘ pfh) * (ε(v) ⊙ (σ_elas∘(ε(u)))); 

### FEA result with the entire domain filled with density 1 elements 
For checking! 

In [ ]:
# Comment/Uncomment for FEA Analysis 
p0 = ones(fem_params.np)
pf_vec = Filter(p0;r, fem_params)
pfh = FEFunction(fem_params.Pf, pf_vec)
pth = (pf -> Threshold(pf; β, η)) ∘ pfh
A_mat = MatrixA(pth; fem_params)
u_vec = stepDisp(fem_params,pth)
uh = FEFunction(fem_params.U0_Disp, u_vec)

writevtk(Ω,"FEAresultsmbb",cellfields=["uh"=>uh,"p0"=>p0,"pth"=>pth]);

### Gradient calculation 
Using adjoint method 

In [ ]:
function gf_pf(pf_vec; β, η, fem_params)
    pfh = FEFunction(fem_params.Pf, pf_vec)
    pth = (pf -> Threshold(pf; β, η)) ∘ pfh
    u_vec = stepDisp(fem_params,pth)
    K_mat = MatrixOf(fem_params)
    u_vec' * K_mat * u_vec
end 

function rrule(::typeof(gf_pf), pf_vec; β, η, fem_params)
    function U_Disp_pullback(dg)
      NO_FIELDS, dg * Dgfdpf(pf_vec; β, η, fem_params) 
    end
    gf_pf(pf_vec; β, η, fem_params), U_Disp_pullback
end

function Dgfdpf(pf_vec; β, η, fem_params)
    pfh = FEFunction(fem_params.Pf, pf_vec)
    pth = (pf -> Threshold(pf; β, η)) ∘ pfh
    A_mat = MatrixA(pth; fem_params)
    u_vec = stepDisp(fem_params,pth)
    O_mat = MatrixOf(fem_params)
    uh = FEFunction(fem_params.U0_Disp, u_vec)
    w_vec =  A_mat' \ (O_mat * u_vec)
    wconjh = FEFunction(fem_params.U0_Disp, w_vec)
    l_temp(dp) = ∫(-2*DAdpf(wconjh,uh, pfh; β, η) * dp)fem_params.dΩ
    dgfdpf = assemble_vector(l_temp, fem_params.Pf)

    return dgfdpf
end 

function pf_p0(p0; r, fem_params)
    pf_vec = Filter(p0; r, fem_params)
    pf_vec
end

function rrule(::typeof(pf_p0), p0; r, fem_params)
  function pf_pullback(dgdpf)
    NO_FIELDS, Dgdp(dgdpf; r, fem_params)
  end
  pf_p0(p0; r, fem_params), pf_pullback
end

function Dgdp(dgdpf; r, fem_params)
    Af = assemble_matrix(fem_params.Pf, fem_params.Qf) do u, v
        ∫(a_f(r, u, v))fem_params.dΩ + ∫(v * u)fem_params.dΩ
    end
    wvec = Af' \ dgdpf
    wh = FEFunction(fem_params.Pf, wvec)
    l_temp(dp) = ∫(wh * dp)fem_params.dΩ
    return assemble_vector(l_temp, fem_params.P)
end 

function gf_p(p0::Vector; r, β, η, fem_params)
    pf_vec = pf_p0(p0; r, fem_params)
    gf_pf(pf_vec; β, η, fem_params)
end

function gf_p(p0::Vector, grad::Vector; r, β, η, fem_params)
    if length(grad) > 0
        dgdp, = Zygote.gradient(p -> gf_p(p; r, β, η, fem_params), p0)
        grad[:] = dgdp
    end
    gvalue = gf_p(p0::Vector; r, β, η, fem_params)
    open("gvaluemma.txt", "a") do io
        write(io, "$gvalue \n")
    end
    gvalue
end; 

### Volume constraint calculation 
Volume should be estimated with filtered densities and not with thresholded densities. 

In [ ]:
function  project(q,model ,dΩ,order)
    reffe = ReferenceFE(lagrangian ,Float64 ,order)
    V = FESpace(model ,reffe ,conformity =:L2)
    a(u,v) =∫(u*v)*dΩ
    b(v) =∫(v*q)*dΩ
    op = AffineFEOperator(a,b,V,V)
    qh = solve(op)
    return  qh
end

In [ ]:
volfrac = 0.3 

dv=get_array(∫(1)fem_params.dΩ)
getpoints = get_cell_points(Ω)

gradc = zeros(fem_params.np)
function cf_p(p0::Vector, gradc::Vector; r, β, η, fem_params)
    if length(gradc) > 0
        gradc[:] = dv
    end
    pf_vec = pf_p0(p0; r, fem_params)
    pfh = FEFunction(fem_params.Pf, pf_vec)
    pth = (pf -> Threshold(pf; β, η)) ∘ pfh
    #pro_pfh = project(pfh⊙dv,model,dΩ,order)
    pthxtr = evaluate(pfh⊙dv,getpoints)
    return sum(pthxtr)[1] - volfrac*sum(dv)
end; 

### MMA to optimize 
Svanberg, Krister. "The method of moving asymptotes—a new method for structural optimization." International journal for numerical methods in engineering 24.2 (1987): 359-373. 

In [ ]:
function gf_p_optimize(p_init; r, β, η, TOL=1e-4, MAX_ITER, fem_params)
    ##################### Optimize #################
    opt = Opt(:LD_MMA, fem_params.np)
    opt.lower_bounds = 0.001
    opt.upper_bounds = 1
    opt.xtol_rel = TOL
    opt.maxeval = MAX_ITER
    opt.min_objective = (p0, grad) -> gf_p(p0, grad; r, β, η, fem_params)
    inequality_constraint!(opt, (p0, gradc) -> cf_p(p0, gradc; r, β, η, fem_params), 1e-8)
    (g_opt, p_opt, ret) = optimize(opt, p_init)
    @show numevals = opt.numevals # the number of function evaluations
    println("got $g_opt at $p_opt after $numevals iterations (returned $ret)")
    return g_opt, p_opt

end; 

### Run the MMA 

In [ ]:
p0 = rand(fem_params.np)
δp = rand(fem_params.np)*1e-8
grad = zeros(fem_params.np)

g0 = gf_p(p0, grad; r, β, η, fem_params)
g1 = gf_p(p0+δp, []; r, β, η, fem_params)
g1-g0, grad'*δp

### Run the MMA 

In [ ]:
# p0 = rand(fem_params.np) # For randomized densities 
grad = zeros(fem_params.np)

p_opt = fill(volfrac, fem_params.np)   # Uniform Initial guess
g_opt = 0

TOL = 1e-5 # tolerance of the MMA 
MAX_ITER = 5000 # Maximum iteration constraints 

g_opt, p_opt = gf_p_optimize(p_opt; r, β, η, TOL, MAX_ITER, fem_params);

###############################################
# For a list of β in the thresholding function 
#----------------------------------------------
# β_list = [4.0,8.0,16.0] 
# for bi = 1 : 3
#     β = β_list[bi]
#     g_opt, p_temp_opt = gf_p_optimize(p_opt; r, β, η, fem_params)
#     global p_opt = p_temp_opt
# end 
# @show g_opt 

### Postprocessing 
Redefine the thresholding function with a sharp increase (i.e., increase β) to get a crisp boundary 

In [ ]:
βpost=64
function Thresholdp(pfh; βpost, η)
    return ((tanh(βpost * η) + tanh(βpost * (pfh - η))) / (tanh(βpost * η) + tanh(βpost * (1.0 - η)))) 
end 

pf_vec = pf_p0(p_opt; r, fem_params)
pfh = FEFunction(fem_params.Pf, pf_vec)
pth = (pf -> Thresholdp(pf; βpost, η)) ∘ pfh; 

### Save the optimized design in png and vtk files 

In [ ]:
fig, ax, plt = plot(fem_params.Ω, pth, colormap = :binary)
Colorbar(fig[1,2], plt)
ax.aspect = AxisAspect(3)
ax.title = "Optimized Design"
# rplot = 110 # Region for plot
limits!(ax, 0, 60, 0, 20)

In [ ]:
save("shapeCantmma.png", fig)

writevtk(Ω,"Cant_result",cellfields= ["p_opt"=>p_opt,"pfh"=>pfh,"pth"=>pth]);

In [ ]:
elapsed_time = time() - t1